
   # One piece Extraction from IMDB


#### Install Library necessary for the Project

In [1]:
# Pandas
%pip install pandas

# Python Dotenv
%pip install python-dotenv

# Beautiful soup
%pip install bs4

# MySQL Connector
%pip install mysql-connector-python

# Selenium
%pip install selenium

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#### Import Library necessary for the Project

In [2]:
# MySQL connector
import mysql.connector

# Web driver 
from selenium import webdriver

# Beautiful soup
from bs4 import BeautifulSoup # this module helps in web scrapping.

# Pandas
import pandas as pd

# importing os module for environment variables
import os

# time
import time
from datetime import datetime

# importing necessary functions from dotenv library
from dotenv import load_dotenv

# loading variables from .env file
load_dotenv() 


True

#### Create the necessary functions

In [3]:
def write_to_file(filename, content, mode='a'):
    """Writes the given log to the log file.

    Args:
        filename (str): The name of the file to add the content.
        content (str): The content to write to the file.
        mode (str, optional): The file opening mode. Defaults to 'a' (append).
            Other modes include:
            - 'w' (write)
            - 'r+' (read and write)
    """

    try:
        with open(filename, mode, encoding='utf-8') as file:
            file.write(str(datetime.now()) + ": " + content + "\n")
    except IOError as e:
        print(f"Error writing to content to log file.': {e}")

In [4]:
def create_soup(url,sleep = 1):
    """Extract data from the url.

    Args:
        url (str): The link to the data to be extracted.
        sleep (int, optional): The time to wait before extracted the page web. Defaults to 1s.
    """
        
    driver = webdriver.Firefox() 
    # Webscrape the URL
    driver.get(url)
    # wait 5s
    time.sleep(sleep)
    # Create a Beautiful soup object
    soup = BeautifulSoup(driver.page_source,"html.parser")
    # close driver
    driver.close()
    return soup

In [5]:
def extract_number_ep():
    """Extract number of data from the beautiful soup.
    """
    soup = create_soup("https://www.imdb.com/title/tt0388629/episodes/?ref_=tt_ep_epl",5)
    number_ep = soup.find("li", class_ = "ipc-inline-list__item sc-a68beae9-0 jefvJo no-text-overflow").text
    number_ep = int(number_ep.split(".E")[1])
    return number_ep


In [6]:
def extract_date(date):
    """Extract date from the beautiful soup..

    Args:
        date (str): String including the date extracted from beautiful soup.
    """
    for i in range(10):
        if "Épisode" in date[i].text:
            date = date[i].text
            date_list=[]
            date_list = date[date.find("le")+3:].split()
            day = int(date_list[0])
            month = int(date_list[1].replace("janv.","01").replace("févr.","02").replace("mars","03").replace("avr.","04").replace("mai","05").replace("juin","06").replace("juil.","07").replace("août","08").replace("sept.","09").replace("oct.","10").replace("nov.","11").replace("déc.","12"))
            year = int(date_list[2])
            return day,month,year

In [7]:
def extract_episod(ep):
    """Extract episod from the beautiful soup..

    Args:
        ep (str): String including the episod extracted from beautiful soup.
    """
    ep_list=[]
    ep_list = ep.split(".E")
    episod = int(ep_list[1])
    return episod

In [8]:
def extract_link(link):
    """Extract link from the beautiful soup..

    Args:
        link (str): String including the link extracted from beautiful soup.
    """
    link = "https://www.imdb.com/" + link[0:link.find("/?ref")]
    return link

In [9]:
def extract_title(title):
    """Extract title from the beautiful soup..

    Args:
        title (str): String including the title extracted from beautiful soup.
    """
    if "," in title:
        title = title.replace(",","")
    return title                 

In [10]:
def create_list(*args):
    """Create list if it doesn't exist
    """
    mydict = {}
    for arg in args:
        if arg not in globals():           
            mydict[arg] = list()
    globals().update(mydict)


#### Connect to the database, create table "IMBD" if it doesn't exist and extract the number of row

In [11]:
# Initialize variables
host = os.getenv("MYSQL_HOSTNAME")
user = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
database = "Onepiece"
table_name = "IMDB"
csv_file = str(datetime.now().year) + "_" + str(datetime.now().month) + "_" + str(datetime.now().day) + "_" + "onepiece.csv"
log_file = str(datetime.now().year) + "_" + str(datetime.now().month) + "_" + str(datetime.now().day) + "_" + "onepiecelog.txt"
url = os.getenv("ONEPIECE_URL")

In [12]:
write_to_file(log_file, f"Connecting to MySQL {database} database","w")

# Connect to the database
try:

    cnx = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database,
        port=3306
    )

    write_to_file(log_file, f"Connected to {database} database")

    # Create a cursor object
    cursor = cnx.cursor()

except mysql.connector.Error as err:
    print(f"Error: {err}")

In [13]:
write_to_file(log_file, f"Creating table IMDB into {database} if doesn't exist")

# Create Table IMDB into MYSQL
try:

    # Create a cursor object
    cursor = cnx.cursor()

    # SQL statement to create the table
    sql = """
    CREATE TABLE IF NOT EXISTS IMDB (
        Episod INT UNIQUE NOT NULL PRIMARY KEY,
        Title VARCHAR(255),
        Day INT,
        Month INT,
        Year INT NOT NULL,
        Rate DOUBLE,
        Link VARCHAR(255)
    )
    """

    # Execute the SQL statement
    cursor.execute(sql)

    # Commit the changes to the database
    cnx.commit()

    write_to_file(log_file, f"Table {table_name} created into {database} successfully")

except mysql.connector.Error as err:
    print(f"Error: {err}")

In [14]:
write_to_file(log_file, f"Extracting number total of episode and number of episod already in IMDB table {database}")

# Extract number of One piece episod

number_tot_ep = extract_number_ep()

write_to_file(log_file, f"There are {number_tot_ep} Onepiece episods at this time")

# Extract number of episod in database
try:
         # Execute a query
    cursor.execute("SELECT COUNT(*) FROM IMDB")
    number_ep_database = cursor.fetchall()
    number_ep_database = int(number_ep_database[0][0])

    write_to_file(log_file, f"There is(are) {number_ep_database} episod(s) already in IMDB table")


except mysql.connector.Error as err:
    print(f"Error: {err}")

number_ep_to_extract = number_tot_ep - number_ep_database

write_to_file(log_file, f"There is(are) {number_ep_to_extract} episod(s) to extract")

if number_ep_to_extract == 0 :
    write_to_file(log_file, f"Program stopped because there is no data to extract")
    raise SystemExit("System stop because there is no episod to extract")

In [15]:
# find the first link to extract

if number_ep_database == 0:
    link  = "https://www.imdb.com/title/tt0947442/?ref_=ttep_ep1"
else:
    # Extract last link in database
    try:
            # Execute a query
        cursor.execute("SELECT link from imdb order by Episod desc limit 1")
        link = cursor.fetchall()
        link = str(link[0][0])
    except mysql.connector.Error as err:
        print(f"Error: {err}")
           
    soup = create_soup(link)
    link = soup.find_all('a', class_='ipc-icon-button sc-3f4e3993-3 iasCTO ipc-icon-button--baseAlt ipc-icon-button--onBase')[1].attrs['href']
    link = extract_link(link)

In [16]:
create_list("episod_list","title_list","rating_list","day_list","month_list","year_list","link_list")

link_list.append(link)

try:

    for i in range(number_ep_to_extract):

        write_to_file(log_file, f"Extracting episod {number_ep_database+1+i} from {link}")
        soup = create_soup(link)
        
        # Season and Episod
        ep = soup.find('div', class_="sc-3f4e3993-0 fYpskP").text
        episod = extract_episod(ep)
        episod_list.append(episod)

        # Title
        title = soup.find('span', class_="hero__primary-text").text
        title_list.append(title)

        #Rating
        rate = soup.find('span', class_="sc-c4ffe080-1 iQZtLP").text.replace(",",".")
        rating_list.append(float(rate))

        # Date
        date = soup.find_all('li', class_="ipc-inline-list__item")
        day,month,year = extract_date(date)
        day_list.append(day)
        month_list.append(month)
        year_list.append(year)

        #Link 
        if i == 0 and number_ep_database == 0: 
            link = soup.find('a', class_='ipc-icon-button sc-3f4e3993-3 iasCTO ipc-icon-button--baseAlt ipc-icon-button--onBase').attrs['href']
        else:
            link = soup.find_all('a', class_='ipc-icon-button sc-3f4e3993-3 iasCTO ipc-icon-button--baseAlt ipc-icon-button--onBase')[1].attrs['href']

        if i < number_ep_to_extract - 1:        
            link = extract_link(link)
            link_list.append(link)

        i = i+1

except WebDriverException as E: 
    write_to_file(log_file, f"Extracting episod {number_ep_database+1+i} from {link} failed")


In [17]:
if len(link_list) > len(episod_list):
    del link_list[-1]

In [18]:
onepiece_data = {"Episod":episod_list, "Title":title_list, "Day":day_list, "Month":month_list, "Year":year_list, "Rate":rating_list, "link":link_list}
onepiece_df = pd.DataFrame(onepiece_data)
onepiece_df.to_csv(csv_file,index=False,mode= "a",header=False)
onepiece_df

,Episod,Title,Day,Month,Year,Rate,link
0,1120,The World Is Shaken! The Ruler's Judgment and ...,22,9,2024,8.6,https://www.imdb.com//title/tt33394990


In [19]:
if onepiece_df.shape[0] == number_ep_to_extract:
    write_to_file(log_file,f"Extracting {number_ep_to_extract} episod successfuly")
else:
    write_to_file(log_file,f"Extraction failed")

In [20]:
write_to_file(log_file,f"Sending {len(onepiece_df)} rows to MySQL table '{table_name}'.")
try:
    # Create a list of column names from the DataFrame
    columns = ', '.join(onepiece_df.columns)

    # Create a placeholders string for the SQL query
    placeholders = ', '.join(['%s'] * len(onepiece_df.columns))

    # Create the SQL query
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    # Execute the SQL query with the DataFrame values
    cursor.executemany(sql, onepiece_df.values.tolist())

    # Commit the changes to the database
    cnx.commit()

    write_to_file(log_file,f"Successfully sent {len(onepiece_df)} rows to MySQL table '{table_name}'.")
except mysql.connector.Error as e:
    print(f"Error sending data to MySQL: {e}")
finally:
    # Close the database cursor and connection
    if cursor:
        cursor.close()
    if cnx:
        cnx.close()